In [1]:
!nvidia-smi

Mon Feb  7 09:53:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# ! gdown --id 1i2x7osfloYkXDHMRn0WuVs2P3m751cj1
# ! pip -q uninstall -y kaggle
# ! pip -q install --upgrade pip
# ! pip -q install kaggle --upgrade
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle competitions download jigsaw-toxic-severity-rating
# ! kaggle competitions download jigsaw-toxic-comment-classification-challenge
# # ! kaggle datasets download aishikai/js-deberta
# ! kaggle datasets download aishikai/js-cleaned-validation-data

In [3]:
# ! unzip /content/jigsaw-toxic-comment-classification-challenge.zip -d JTCC
# ! rm /content/jigsaw-toxic-comment-classification-challenge.zip
# ! unzip /content/jigsaw-toxic-severity-rating.zip -d Data
# ! rm /content/jigsaw-toxic-severity-rating.zip
# # ! unzip /content/js-deberta.zip -d deberta
# # ! rm /content/js-deberta.zip
# ! unzip /content/js-cleaned-validation-data.zip -d JTSR
# ! rm /content/js-cleaned-validation-data.zip

unzip:  cannot find or open /content/jigsaw-toxic-comment-classification-challenge.zip, /content/jigsaw-toxic-comment-classification-challenge.zip.zip or /content/jigsaw-toxic-comment-classification-challenge.zip.ZIP.
rm: cannot remove '/content/jigsaw-toxic-comment-classification-challenge.zip': No such file or directory
unzip:  cannot find or open /content/jigsaw-toxic-severity-rating.zip, /content/jigsaw-toxic-severity-rating.zip.zip or /content/jigsaw-toxic-severity-rating.zip.ZIP.
rm: cannot remove '/content/jigsaw-toxic-severity-rating.zip': No such file or directory
unzip:  cannot find or open /content/js-cleaned-validation-data.zip, /content/js-cleaned-validation-data.zip.zip or /content/js-cleaned-validation-data.zip.ZIP.
rm: cannot remove '/content/js-cleaned-validation-data.zip': No such file or directory


In [4]:
# !unzip /content/JTCC/sample_submission.csv.zip -d JTCC
# !rm /content/JTCC/sample_submission.csv.zip
# !unzip /content/JTCC/test.csv.zip -d JTCC
# !rm /content/JTCC/test.csv.zip
# !unzip /content/JTCC/test_labels.csv.zip -d JTCC
# !rm /content/JTCC/test_labels.csv.zip
# !unzip /content/JTCC/train.csv.zip -d JTCC
# !rm /content/JTCC/train.csv.zip

unzip:  cannot find or open /content/JTCC/sample_submission.csv.zip, /content/JTCC/sample_submission.csv.zip.zip or /content/JTCC/sample_submission.csv.zip.ZIP.
rm: cannot remove '/content/JTCC/sample_submission.csv.zip': No such file or directory
unzip:  cannot find or open /content/JTCC/test.csv.zip, /content/JTCC/test.csv.zip.zip or /content/JTCC/test.csv.zip.ZIP.
rm: cannot remove '/content/JTCC/test.csv.zip': No such file or directory
unzip:  cannot find or open /content/JTCC/test_labels.csv.zip, /content/JTCC/test_labels.csv.zip.zip or /content/JTCC/test_labels.csv.zip.ZIP.
rm: cannot remove '/content/JTCC/test_labels.csv.zip': No such file or directory
unzip:  cannot find or open /content/JTCC/train.csv.zip, /content/JTCC/train.csv.zip.zip or /content/JTCC/train.csv.zip.ZIP.
rm: cannot remove '/content/JTCC/train.csv.zip': No such file or directory


In [5]:
# ! pip -q install sentencepiece
# ! pip -q install transformers
# ! pip -q install wandb --upgrade
# ! pip -q install colorama

In [6]:
import os
import gc
import copy
import time
import random
import string
import wandb
wandb.login()

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score,accuracy_score

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW, AutoConfig

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

wandb: Currently logged in as: aishikai (use `wandb login --relogin` to force relogin)


In [7]:
df = pd.read_csv("/content/JTCC/train.csv")

In [8]:
Config = {"seed": 2021,
          "csv_path":"/content/JTCC/train.csv",
          "epochs": 6,
          "model_name": "sentence-transformers/all-mpnet-base-v2",
          "train_batch_size": 96,
          "valid_batch_size": 96,
          "num_workers": 4,
          "max_length": 128,
          "lr": 1e-5,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_0":20,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          'wandb': True,
          'apex': False,
          'n_accum': 1,
          "debug":False,
          "column": 'toxic'
          }

Config["tokenizer"] = AutoTokenizer.from_pretrained(Config['model_name'])

In [9]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(Config['seed'])

In [10]:
# Start W&B logging
if Config['wandb']:
    run = wandb.init(
        project='pytorch',
        config=Config,
        group='nlp',
        job_type='train',
        anonymous='must'
    )

In [11]:
# Some utility functions
def wandb_log(**kwargs):
    """
    Logs a key-value pair to W&B
    """
    for k, v in kwargs.items():
        wandb.log({k: v})

In [12]:
def create_folds(data, num_splits = Config["n_fold"]):
    data["kfold"] = -1
    kf = StratifiedKFold(n_splits=num_splits)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data[Config.get('column')].values)):
        data.loc[v_, 'kfold'] = f
    return data

def map_cls_wt(x):
    if x:
        return weight[1]
    else:
        return weight[0]

In [13]:
class JigsawDataset(Dataset):
    def __init__(self, df, tokenizer=Config["tokenizer"], max_length=Config["max_length"]):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.comment_text = df['comment_text'].values
        self.target = df[Config.get('column')].values
        self.weight = df["weight"].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        comment_text = self.comment_text[index]
        target = self.target[index]
        weight = self.weight[index]
        inputs = self.tokenizer.encode_plus(
                                comment_text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
        input_ids = inputs['input_ids']
        input_mask = inputs['attention_mask']
        
        
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'input_mask': torch.tensor(input_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.long),
            'weight': torch.tensor(weight, dtype=torch.float)
        }

In [14]:
class JigsawModel(nn.Module):
    def __init__(self, model_name):
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(768,256)
        self.fc2 = nn.Linear(256, Config['num_classes'])
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        out = self.fc1(out)
        outputs = torch.sigmoid(self.fc2(out))
        return outputs

In [15]:
# class AttentionHead(nn.Module):
#     def __init__(self, in_features, hidden_dim):
#         super().__init__()
#         self.in_features = in_features
#         self.middle_features = hidden_dim
#         self.W = nn.Linear(in_features, hidden_dim)
#         self.V = nn.Linear(hidden_dim, 1)
#         self.out_features = hidden_dim

#     def forward(self, features):
#         att = torch.tanh(self.W(features))
#         score = self.V(att)
#         attention_weights = torch.softmax(score, dim=1)
#         context_vector = attention_weights * features
#         context_vector = torch.sum(context_vector, dim=1)

#         return context_vector

# class JigsawModel(nn.Module):
#     def __init__(self, model_name, config_path=None, pretrained=False):
#         super().__init__()
#         self.config = AutoConfig.from_pretrained(model_name, output_hidden_states=True)
#         self.model = AutoModel.from_pretrained(model_name, config=self.config)
#         self.head = AttentionHead(self.config.hidden_size,self.config.hidden_size)
#         self.dropout = nn.Dropout(0.4)
#         self.fc1 = nn.Linear(self.config.hidden_size,256)
#         self.fc2 = nn.Linear(256,1)

#     def forward(self, ids, mask):
#         x = self.model(input_ids=ids,attention_mask=mask)[0]
#         x = self.head(x)
#         x = self.dropout(x)
#         x = self.fc1(x)
#         x = torch.sigmoid(self.fc2(x))
#         return x

In [16]:
def criterion(outputs, targets, weight=None):
        return nn.BCELoss(weight = weight)(outputs.view(-1), targets)

In [17]:
class Trainer:
    def __init__(self, config, dataloaders, optimizer, model, loss_fns, scheduler, device=Config['device'], apex=Config['apex']):
        self.train_loader, self.valid_loader = dataloaders
        self.train_loss_fn, self.valid_loss_fn = loss_fns
        self.scheduler = scheduler
        self.optimizer = optimizer
        self.model = model
        self.device = device
        self.apex = apex
        self.Config = config
    
    def train_one_epoch(self):
        if self.apex:
            scaler = GradScaler()

        self.model.train()
        train_pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader))
        train_preds, train_labels = [], []
        dataset_size = 0
        running_loss = 0.0

        for step, data in train_pbar:        
            input_ids = data['input_ids'].to(self.device, dtype = torch.long)
            input_mask = data['input_mask'].to(self.device, dtype = torch.long)
            targets = data['target'].to(self.device, dtype=torch.float)
            weight = data['weight'].to(self.device, dtype=torch.float)

            batch_size = input_ids.size(0)

            if self.apex:
                with(autocast(enabled=True)):

                    outputs = model(input_ids, input_mask)
                    
                    loss = criterion(outputs, targets, weight)
                    loss = loss / self.Config["n_accum"]
                
                scaler.scale(loss).backward()

                if (step + 1) % self.Config["n_accum"] == 0:
                    scaler.step(self.optimizer)
                    scaler.update()
                    optimizer.zero_grad()

                    if self.scheduler:
                        self.scheduler.step()

                running_loss += (loss.item() * batch_size)
                dataset_size += batch_size
            
            else:
                outputs = model(input_ids, input_mask)
                loss = criterion(outputs, targets, weight)
                loss = loss / self.Config["n_accum"]

                loss.backward()
                loss = loss / self.Config["n_accum"]
                if (step + 1)%self.Config["n_accum"] == 0:
                    optimizer.step()
                    optimizer.zero_grad()

                    if self.scheduler:
                        self.scheduler.step()

                running_loss += (loss.item() * batch_size)
                dataset_size += batch_size

            epoch_loss = running_loss / dataset_size                
            
            train_pbar.set_postfix(Train_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])   
        
        gc.collect()
        
        return epoch_loss

    @torch.no_grad()
    def valid_one_epoch(self):

        self.model.eval()
        valid_preds = []
        valid_targets = []
        valid_pbar = tqdm(enumerate(self.valid_loader), total=len(self.valid_loader))
        dataset_size = 0
        running_loss = 0.0

        for step, data in valid_pbar:
            input_ids = data['input_ids'].to(self.device, dtype = torch.long)
            input_mask = data['input_mask'].to(self.device, dtype = torch.long)
            targets = data['target'].to(self.device, dtype=torch.float)
            weight = data['weight'].to(self.device, dtype=torch.float)

            batch_size = input_ids.size(0)

            outputs = model(input_ids, input_mask)
            loss = criterion(outputs, targets)
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size
            epoch_loss = running_loss / dataset_size
            
            valid_preds.extend(outputs.cpu().detach().numpy())
            valid_targets.extend(targets.cpu().detach().numpy())

            valid_pbar.set_postfix(Valid_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])
            
            
        
            gc.collect()

        
        valid_preds = np.concatenate(valid_preds)
        # valid_targets = np.concatenate(valid_targets)
        return valid_preds, valid_targets, epoch_loss
            

    def fit(self, fold: str, epochs: int = 10, output_dir: str = "/content/models/", custom_name: str = 'model.pth'):
        """
        Low-effort alternative for doing the complete training and validation process
        """
        best_loss = int(1e+7)
        for epx in range(epochs):
            print(f"{'='*20} Epoch: {epx+1} / {epochs} {'='*20}")

            train_loss = self.train_one_epoch()
            print(f"Training loss: {train_loss:.4f}")

            valid_preds, valid_targets, valid_loss = self.valid_one_epoch()
            print(f"Validation loss: {valid_loss:.4f}")
            prediction = valid_preds > 0.5

            f1 = f1_score(valid_targets, prediction)
            print(f"Valid f1_score : {f1:.4f}")

            if valid_loss < best_loss:
                best_loss = valid_loss
                self.save_model(output_dir, custom_name)
                print(f"Saved model with val_loss: {best_loss:.4f}")
            

            # Log
            if Config['wandb']:
                wandb_log(
                    train_loss=train_loss,
                    val_loss=valid_loss
                )

    def save_model(self, path, name, verbose=False):
        """
        Saves the model at the provided destination
        """
        try:
            if not os.path.exists(path):
                os.makedirs(path)
        except:
            print("Errors encountered while making the output directory")

        torch.save(self.model.state_dict(), os.path.join(path, name))
        if verbose:
            print(f"Model Saved at: {os.path.join(path, name)}")

In [18]:
if __name__ == '__main__':
    train_file = pd.read_csv(Config['csv_path'])
    # for col in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    for col in ['toxic']:
        print (f"{'='*10}{col}{'='*10}")
        Config['column'] = col

        if Config["debug"]:
            train_file = train_file.sample(n=100).reset_index(drop=True)

        train_file = create_folds(train_file)
            
        for fold_ in range(Config["n_fold"]):
            print(f"{'='*40} Fold: {fold_+1} / {Config['n_fold']} {'='*40}")
            
            train_ = train_file.loc[train_file["kfold"]!=fold_]
            valid_ = train_file.loc[train_file["kfold"]==fold_]

            weight = compute_class_weight(class_weight='balanced',classes = np.unique(train_[col].values), y=train_[col].values)
            train_["weight"] = train_[col].map(map_cls_wt)
            valid_["weight"] = valid_[col].map(map_cls_wt)

            
            train_set = JigsawDataset(
                df = train_
            )
            valid_set = JigsawDataset(
                df = valid_
            )
            
            train_loader = DataLoader(
                train_set,
                batch_size = Config['train_batch_size'],
                shuffle = True,
                num_workers = Config['num_workers'],
                pin_memory = True
            )
            
            valid_loader = DataLoader(
                valid_set,
                batch_size = Config['valid_batch_size'],
                shuffle = False,
                num_workers = Config['num_workers'],
            )
            
            model = JigsawModel(model_name=Config['model_name'])
            model = model.to(torch.device(Config['device']))
            if Config['wandb']:
                wandb.watch(model)
                
            optimizer = torch.optim.AdamW(model.parameters(), lr=Config['lr'])
            scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, 
                T_0=Config['T_0'], 
                eta_min=Config['min_lr']
            )
            train_lfn, valid_lfn = criterion, criterion
            
            trainer = Trainer(
                config = Config,
                dataloaders=(train_loader, valid_loader),
                loss_fns=(train_lfn, valid_lfn),
                optimizer=optimizer,
                model = model,
                scheduler=scheduler,
                apex=Config['apex'],
            )
            
            trainer.fit(
                fold = fold_,
                epochs = Config['epochs'],
                custom_name = f"{Config['model_name'].split('/')[-1]}_{Config['column']}_{fold_}_model.bin"
            )

==========toxic==========
======================================== Fold: 1 / 5 ========================================
==================== Epoch: 1 / 6 ====================


100%|██████████| 1995/1995 [16:39<00:00,  2.00it/s, LR=2.32e-6, Train_Loss=0.202]


Training loss: 0.2024


100%|██████████| 499/499 [02:49<00:00,  2.94it/s, LR=2.32e-6, Valid_Loss=0.185]


Validation loss: 0.1850
Valid f1_score : 0.7223
Saved model with val_loss: 0.1850
==================== Epoch: 2 / 6 ====================


...

Validation loss: 0.1314
Valid f1_score : 0.7843
==================== Epoch: 6 / 6 ====================


100%|██████████| 1995/1995 [16:29<00:00,  2.02it/s, LR=5.5e-6, Train_Loss=0.0677]

Training loss: 0.0677



100%|██████████| 499/499 [02:45<00:00,  3.01it/s, LR=5.5e-6, Valid_Loss=0.166]


Validation loss: 0.1662
Valid f1_score : 0.7719


In [20]:
!zip /content/drive/MyDrive/JS_p0.zip /content/models/all-mpnet-base-v2_toxic_0_model.bin \
/content/models/all-mpnet-base-v2_toxic_1_model.bin \
/content/models/all-mpnet-base-v2_toxic_2_model.bin \
/content/models/all-mpnet-base-v2_toxic_3_model.bin \
/content/models/all-mpnet-base-v2_toxic_4_model.bin 

  adding: content/models/all-mpnet-base-v2_toxic_0_model.bin (deflated 8%)
  adding: content/models/all-mpnet-base-v2_toxic_1_model.bin (deflated 8%)
  adding: content/models/all-mpnet-base-v2_toxic_2_model.bin (deflated 8%)
  adding: content/models/all-mpnet-base-v2_toxic_3_model.bin (deflated 8%)
  adding: content/models/all-mpnet-base-v2_toxic_4_model.bin (deflated 8%)


In [20]:
# !zip /content/drive/MyDrive/JS_p1.zip /content/models/deberta-v3-base_severe_toxic_0_model.bin \
# /content/models/deberta-v3-base_severe_toxic_1_model.bin \
# /content/models/deberta-v3-base_severe_toxic_2_model.bin \
# /content/models/deberta-v3-base_severe_toxic_3_model.bin \
# /content/models/deberta-v3-base_severe_toxic_4_model.bin 

In [21]:
# !zip /content/drive/MyDrive/JS_p2.zip /content/models/deberta-v3-base_threat_0_model.bin \
# /content/models/deberta-v3-base_threat_1_model.bin \
# /content/models/deberta-v3-base_threat_2_model.bin \
# /content/models/deberta-v3-base_threat_3_model.bin \
# /content/models/deberta-v3-base_threat_4_model.bin 

In [22]:
# !zip /content/drive/MyDrive/JS_p3.zip /content/models/deberta-v3-base_insult_0_model.bin \
# /content/models/deberta-v3-base_insult_1_model.bin \
# /content/models/deberta-v3-base_insult_2_model.bin \
# /content/models/deberta-v3-base_insult_3_model.bin \
# /content/models/deberta-v3-base_insult_4_model.bin 

In [23]:
# !zip /content/drive/MyDrive/JS_p4.zip /content/models/deberta-v3-base_obscene_0_model.bin \
# /content/models/deberta-v3-base_obscene_1_model.bin \
# /content/models/deberta-v3-base_obscene_2_model.bin \
# /content/models/deberta-v3-base_obscene_3_model.bin \
# /content/models/deberta-v3-base_obscene_4_model.bin 

In [24]:
# !zip /content/drive/MyDrive/JS_p5.zip /content/models/deberta-v3-base_identity_hate_0_model.bin \
# /content/models/deberta-v3-base_identity_hate_1_model.bin \
# /content/models/deberta-v3-base_identity_hate_2_model.bin \
# /content/models/deberta-v3-base_identity_hate_3_model.bin \
# /content/models/deberta-v3-base_identity_hate_4_model.bin 

In [25]:
# !rm -r /content/models

In [26]:
MODEL_PATHS = [f"/content/deberta/JS_deberta-base-multi-cat/JS_deberta-base-multi-cat/deberta-v3-base_toxic_{fold}_model.bin" for  fold in range(5)]

In [33]:
val_df = pd.read_csv("/content/JTSR/Validation_data_clean_no_duplicates.csv")
val_df.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [34]:
class JigsawDatasetTest(Dataset):
    def __init__(self, df, tokenizer, max_length, col):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df[col].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
                        text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']        
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
        }

In [35]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = JigsawModel(Config["model_name"])
        model.to(Config['device'])
        model.load_state_dict(torch.load(path, map_location = device))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [36]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        PREDS.append(outputs.sigmoid().view(-1).cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    del model
    gc.collect()
    
    return PREDS

In [37]:
test_dataset = JigsawDatasetTest(val_df, Config['tokenizer'], max_length=Config['max_length'], col = "more_toxic")
test_loader = DataLoader(test_dataset, batch_size=Config['valid_batch_size'],
                            num_workers=2, shuffle=False, pin_memory=True)
preds_more = inference(MODEL_PATHS , test_loader, Config['device'])

Downloading:   0%|          | 0.00/354M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 154/154 [01:12<00:00,  2.13it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

Getting predictions for model 2


...

100%|██████████| 154/154 [01:12<00:00,  2.13it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

Getting predictions for model 4


100%|██████████| 154/154 [01:12<00:00,  2.13it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

Getting predictions for model 5


100%|██████████| 154/154 [01:12<00:00,  2.13it/s]


In [38]:
test_dataset = JigsawDatasetTest(val_df, Config['tokenizer'], max_length=Config['max_length'], col = "less_toxic")
test_loader = DataLoader(test_dataset, batch_size=Config['valid_batch_size'],
                            num_workers=2, shuffle=False, pin_memory=True)
preds_less = inference(MODEL_PATHS , test_loader, Config['device'])

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 154/154 [01:12<00:00,  2.13it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

Getting predictions for model 2


100%|██████████| 154/154 [01:12<00:00,  2.13it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

...

100%|██████████| 154/154 [01:12<00:00,  2.13it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

Getting predictions for model 4


100%|██████████| 154/154 [01:12<00:00,  2.12it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

Getting predictions for model 5


100%|██████████| 154/154 [01:15<00:00,  2.05it/s]


In [40]:
predictions = preds_more > preds_less
val_df["truth"] = 1
val_truths = val_df.truth.values
print(f"Accuracy: {accuracy_score(val_truths, predictions)}")

Accuracy: 0.7755562721702677
